In [1]:
from bokeh.plotting import output_notebook, show, figure

In [2]:
output_notebook()

Loading BokehJS ...

In [6]:
f = figure()
f.line([1,2,3,4,5,6,7], [2,4,6,8,10,12,14])
show(f)

In [7]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [8]:
init_notebook_mode(connected=True)

In [9]:
iplot([{"x": [1, 2, 3], "y": [3, 1, 6]}])